# Format Europe energy demand

Data are hourly, country aggregates, from ENTSOE: https://www.entsoe.eu/data/power-stats/

In [152]:
import pandas as pd
import numpy as np

In [153]:
data_path = "/g/data/w42/dr6273/work/data/ENTSOE/"

### 2006 - 2015 data

In [154]:
hourly_06_15 = pd.read_csv(
    data_path+"Monthly-hourly-load-values_2006-2015.csv",
    skiprows=3,
    header=0,
    parse_dates={'timestamp': ['Year','Month', "Day"]},
    index_col="timestamp",
)

In [155]:
hourly_06_15

,Country,Coverage ratio,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
timestamp,,,,,,,,,,,,,,,,,,,,,
2006-01-01,AT,100.0,6297.0,6005.0,5743.0,5397.0,5213.0,5190.0,4916.0,5158.0,...,6247.0,6231.0,6603.0,6979.0,7022.0,6930.0,6560.0,6170.0,6501.0,6168.0
2006-01-02,AT,100.0,5900.0,5643.0,5524.0,5372.0,5481.0,5941.0,6459.0,7352.0,...,8190.0,8168.0,8469.0,8735.0,8617.0,8218.0,7773.0,7247.0,7413.0,6926.0
2006-01-03,AT,100.0,6483.0,6192.0,5991.0,5773.0,5849.0,6252.0,6685.0,7480.0,...,8112.0,8055.0,8342.0,8670.0,8534.0,8203.0,7745.0,7214.0,7397.0,6981.0
2006-01-04,AT,100.0,6522.0,6217.0,6027.0,5802.0,5866.0,6289.0,6730.0,7554.0,...,8145.0,8081.0,8331.0,8470.0,8460.0,8185.0,7759.0,7242.0,7441.0,6981.0
2006-01-05,AT,100.0,6575.0,6237.0,6065.0,5841.0,5887.0,6283.0,6704.0,7491.0,...,8046.0,8042.0,8268.0,8361.0,8453.0,8114.0,7606.0,7139.0,7333.0,6888.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-27,UA_W,100.0,534.0,523.0,511.0,524.0,537.0,566.0,612.0,662.0,...,669.0,719.0,783.0,779.0,771.0,726.0,721.0,663.0,582.0,547.0
2015-12-28,UA_W,100.0,510.0,521.0,519.0,529.0,551.0,637.0,744.0,803.0,...,805.0,851.0,885.0,875.0,868.0,834.0,782.0,724.0,642.0,601.0
2015-12-29,UA_W,100.0,550.0,535.0,538.0,545.0,597.0,685.0,762.0,813.0,...,797.0,850.0,905.0,889.0,878.0,847.0,828.0,763.0,692.0,635.0


### Important!

Some countries don't have the full range of data. E.g. GB starts in 2010, IE in 2008. So, let's average across regions instead of sum.

Sum over hours

In [156]:
daily_06_15 = hourly_06_15[["Country", "Coverage ratio"]]

In [157]:
daily_06_15["Unscaled power (MWh/day)"] = hourly_06_15.iloc[:,2:].sum(axis=1)

/jobfs/94100169.gadi-pbs/ipykernel_1915945/3514301491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_06_15["Unscaled power (MWh/day)"] = hourly_06_15.iloc[:,2:].sum(axis=1)


According to the data, "To get the real value, you need to scale them to 100% for coverage ratio less than 100."

In [158]:
def scale(ratio, value):
    """
    Scale the value to 100% coverage
    
    ratio: coverage ration (%)
    value: value to scale to 100
    """
    return value / (ratio / 100) 

In [159]:
daily_06_15["Power (MWh/day)"] = daily_06_15.apply(
    lambda x : scale(x['Coverage ratio'], x["Unscaled power (MWh/day)"]), axis=1
)

/jobfs/94100169.gadi-pbs/ipykernel_1915945/1548445770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_06_15["Power (MWh/day)"] = daily_06_15.apply(


In [160]:
daily_06_15 = daily_06_15.drop("Unscaled power (MWh/day)", axis=1)
daily_06_15 = daily_06_15.drop("Coverage ratio", axis=1)

Only keep the following countries: Austria, Belgium, Denmark, France, Germany, Ireland, Italy, Luxembourg, the Netherlands, Norway, Portugal, Spain, Sweden, Switzerland and the UK.

In [161]:
daily_06_15["Country"].unique()

array(['AT', 'BA', 'BE', 'BG', 'CH', 'CS', 'CY', 'CZ', 'DE', 'DK', 'DK_W',
       'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IS', 'IT',
       'LT', 'LU', 'LV', 'ME', 'MK', 'NI', 'NL', 'NO', 'PL', 'PT', 'RO',
       'RS', 'SE', 'SI', 'SK', 'UA_W'], dtype=object)

In [162]:
# codes = ["AT", "BE", "DK", "FR", "DE", "IE", "IT", "LU", "NL", "NO", "PT", "ES", "SE", "CH", "GB"]
# len(codes)

In [163]:
# daily = daily.loc[daily["Country"].isin(codes)]

Years available for each country

In [164]:
# set_2006 = [i for i in codes if i not in ["DK", "GB", "IE", "NO", "SE"]]
# set_2006

Average over all countries

In [165]:
# daily = daily.groupby(daily.index).mean(numeric_only=True)

In [166]:
# daily_2006_2015 = daily.loc[daily["Country"].isin(set_2006)]
# daily_2006_2015 = daily_2006_2015.groupby(daily_2006_2015.index).sum(numeric_only=True)

In [167]:
# daily_2010_2015 = daily.loc[daily.index >= "2010"]
# daily_2010_2015 = daily_2010_2015.groupby(daily_2010_2015.index).sum(numeric_only=True)

In [168]:
# daily_2006_2015.head()

In [169]:
# daily_2010_2015.head()

In [170]:
# daily_2006_2015.plot()

In [171]:
# daily_2010_2015.plot()

In [172]:
# daily.to_csv(
#     data_path+"EU_daily_avg_demand_2006-2015.csv"
# )

In [173]:
# daily_2006_2015.to_csv(
#     data_path+"EU_subset_daily_demand_2006-2015.csv"
# )

In [174]:
# daily_2010_2015.to_csv(
#     data_path+"EU_daily_demand_2010-2015.csv"
# )

### 2016-19 data

In [175]:
def prep_15_18_data():
    """
    Prepare the demand data for 2015-2017 and 2018-2019
    """
    def _load(path):
        df = pd.read_csv(path)
        df = df.dropna()
        df["timestamp"] = df["DateShort"] + df["TimeFrom"]
        df["timestamp"] = pd.to_datetime(df["timestamp"], format="%d/%m/%Y%H:%M")
        df = df.set_index("timestamp")
        df = df[["CountryCode", "Value_ScaleTo100"]]
        return df
    
    df_15_17 = _load(data_path+"MHLV_data-2015-2017.csv")
    df_18_19 = _load(data_path+"MHLV_data-2018-2019.csv")
    
    df = pd.concat([df_15_17, df_18_19])
    
    df = df.groupby("CountryCode").resample("D").sum(numeric_only=True)
    df = df.reset_index(level="CountryCode")
    df.columns = ["Country", "Power (MWh/day)"]
    return df

In [176]:
df_15_18 = prep_15_18_data()

In [177]:
df_15_18

,Country,Power (MWh/day)
timestamp,,
2016-12-31,AL,1000.00
2017-01-01,AL,22320.00
2017-01-02,AL,22100.00
2017-01-03,AL,23290.00
2017-01-04,AL,23980.00
...,...,...
2019-07-27,TR,882028.25
2019-07-28,TR,801862.89
2019-07-29,TR,935338.60


### Concat these

Countries common to both time periods

In [178]:
common_countries = [i for i in np.unique(daily_06_15["Country"]) if i in np.unique(df_15_18["Country"])]

In [180]:
print(common_countries)

['AT', 'BA', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IS', 'IT', 'LT', 'LU', 'LV', 'ME', 'MK', 'NL', 'NO', 'PL', 'PT', 'RO', 'RS', 'SE', 'SI', 'SK']


No restriction on starting year

In [181]:
df_06_15 = daily_06_15.loc[daily_06_15["Country"].isin(common_countries)]

In [182]:
df_06_15.groupby("Country").apply(lambda x: (x.index[0], x.index[-1]))

Country
AT    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
BA    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
BE    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
BG    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
CH    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
CY    (2010-01-01 00:00:00, 2015-12-31 00:00:00)
CZ    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
DE    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
DK    (2010-01-01 00:00:00, 2015-12-31 00:00:00)
EE    (2009-01-01 00:00:00, 2015-12-31 00:00:00)
ES    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
FI    (2010-01-01 00:00:00, 2015-12-31 00:00:00)
FR    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
GB    (2010-01-01 00:00:00, 2015-12-31 00:00:00)
GR    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
HR    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
HU    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
IE    (2008-01-01 00:00:00, 2015-12-31 00:00:00)
IS    (2008-01-01 00:00:00, 2015-12-31 00:00:00)
IT    (2006-01-01 00:00:00, 2015-12-31 00:00:00)
LT    (2010-

But make sure to remove end of 2015 days from 2015-2019 data, and remove 2019 as some countries incomplete

In [183]:
df_15_18 = df_15_18.loc[df_15_18["Country"].isin(common_countries)]

In [184]:
df_15_18 = df_15_18[
    (df_15_18.index.year > 2015) &
    (df_15_18.index.year < 2019)
]

In [185]:
df_15_18.groupby("Country").apply(lambda x: (x.index[0], x.index[-1]))

Country
AT    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
BA    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
BE    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
BG    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
CH    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
CY    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
CZ    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
DE    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
DK    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
EE    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
ES    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
FI    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
FR    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
GB    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
GR    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
HR    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
HU    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
IE    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
IS    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
IT    (2016-01-01 00:00:00, 2018-12-31 00:00:00)
LT    (2016-

In [186]:
pd.concat([
    daily_06_15.loc[daily_06_15["Country"].isin(common_countries)],
    df_15_18.loc[df_15_18["Country"].isin(common_countries)]
])

,Country,Power (MWh/day)
timestamp,,
2006-01-01,AT,145451.00
2006-01-02,AT,176630.00
2006-01-03,AT,179083.00
2006-01-04,AT,179265.00
2006-01-05,AT,178663.00
...,...,...
2018-12-27,SK,74862.37
2018-12-28,SK,74492.39
2018-12-29,SK,73523.42


In [187]:
df = pd.concat([df_06_15, df_15_18])

In [188]:
df = df.groupby("Country").resample("D").sum(numeric_only=True).reset_index(level="Country")

In [189]:
df

,Country,Power (MWh/day)
timestamp,,
2006-01-01,AT,145451.00
2006-01-02,AT,176630.00
2006-01-03,AT,179083.00
2006-01-04,AT,179265.00
2006-01-05,AT,178663.00
...,...,...
2018-12-27,SK,74862.37
2018-12-28,SK,74492.39
2018-12-29,SK,73523.42


In [190]:
df.to_csv(
    data_path+"ENTSOE_daily_demand_2006-2018.csv"
)